In [84]:
# MOPTA Competition Base Model 04/13/2024

############ Initialize Tools ############
begin
    import Pkg;
    # Initialize JuMP to allow mathematical programming models
    # Add Packages if you are running this for the first time


    # Pkg.add("JuMP")
    # Pkg.add("CSV")
    # Pkg.add("DataFrames")
    # Pkg.add("Clp")
    # Pkg.add("PlotlyJS")
    # Pkg.add("Dates")
    # Pkg.add("XLSX")
    # Pkg.add("FileIO")
    # Pkg.add("PrettyTables")
    # Pkg.add("Gurobi")
    # Pkg.add("PyCall")


    using JuMP
    using CSV
    using DataFrames
    using PlotlyJS
    using Dates
    using XLSX
    using FileIO
    using Base
    using PrettyTables
    using WebIO
    using Gurobi
    # using Clp
    # using Ipopt
end

############ Program Preparations ############
begin
    # Update automatically the date when this program is run.
    today_date = today()

    # Please update information of this program to automatically update the code name.
    code_name = "BaseModel"
    version = "1.0"

    folder_name = "$code_name._V$version._$today_date"

    # Create folder to later save data and plots
    begin
        # Define the path for the new folder
        folder_path = "$folder_name"

        # Use mkpath() to create the folder if it doesn't exist
        mkpath(folder_path)
    end

    # Function to save a plot as a PNG file in the specified folder
    function save_plot(plot, path, filename, format="png")
        # Create the full file path with the specified filename and format
        full_path = joinpath(path, string(filename, ".", format))

        # Save the plot as an image in the desired format
        savefig(plot, full_path)
    end
end

############ Load Data ############
begin
    scenarios = Dict()
    for i in 1:10
        file_path = "data/yearly_data_$(i).csv"
        scenarios[i] = CSV.read(file_path, DataFrame)
    end

end

############ Declare Parameters ############
begin
    # Time Horizon Parameters
    begin
        TimeStart = 1;
        # TimeEnd = 365 * 24 * 4;
        TimeEnd = 7 * 24 * 4;
        TIME = collect(TimeStart:1:TimeEnd); # Collect timesteps into a vector
        NumTime = length(TIME); # Number of timesteps, useful for indexing
        stepsize = 15;
        δt = stepsize/60; # [hr] Declare stepzize for the optimization program
    end
    # Energy Generation Parameters
    begin
        # Solar PV Parameters
        C_PV = 400 # [$/kW] Capital Cost of Solar PV
        C_PV_OP = 10 # [$/(kW*YR)] Operational and Maintenance Cost of Solar PV
        η_PVIV = 0.96 # [1] PV(DC) to Home(AC) inverter efficiency

        # Wind Parameters
        C_W = 750 # [$/kW] Capital Cost of Wind Generation
        C_W_OP = 45 # [$/(kW*YR)] Operational and Maintenance Cost of Wind Generation
    end
    # Electrolyzer Parameters
    begin
        C_EL = 1000 # [$/kW]
        α_EL = 0.7 # electricity to hydrogen
        k_E2H = (1/0.7)/50 # [kwh electricity to kg hydrogen]
    end
    # Fuel Cell Parameters
    begin
        C_FC = 200 # [$/kW]
        α_FC = 0.75 # hydrogen to electricity
        k_H2E = (1/0.75)*33 # [kg hydrogen to kWh electricity]
    end
    # Storage Parameters
    begin
        L_ss = 0.01 # [/hr] short term leakage rate
        L_ls = 0.03/24 # [/hr] long term leakage rate
        β_l2g = 0.75 # liquid to gas efficiency
        β_g2l = 0.9 # gas to liquid efficiency

        C_l2g = 0 # [$/kg] liquid to gas conversion cost
        C_g2l = 2.75 # [$/kg] gas to liquid conversion cost

        C_ss = 0.33 # [$/kg] gas hydrogen
        C_ls = 1.2 # [$/kg] liquid hydrogen storage
        C_C_ss = 1000 # [$/kg] capacity cost of hydrogen gas storage
        C_C_ls = 1400 # [$/kg] capacity cost of hydrogen liquid storage
    end
    # Transmission & Distribution Parameters
    begin
        μ = 0.995 # transportation efficiency
        C_d = 10 # [$/kg] Distribution cost of hydrogen
    end
    # Economic Parameters
    begin
        Lifetime = 20 # [YR]
        d = 0.03 # [1] Discount Rate
        CRF = (d*(1+d)^Lifetime)/((1+d)^Lifetime-1) # [1] Capital Recovery Factor
    end
    begin
    # Capacity Parameteres
        UB_WindSize = Inf
        UB_ESize = Inf
    end
end

function Optimize(stepsize)
    ########## Instructions  ##########
    begin
    end
    ########## Declare model  ##########
    begin
        # Define the model name and solver. In this case, model name is "m"
        # m = Model(Clp.Optimizer)
        # m = Model(Ipopt.Optimizer)
        # For Gurobi (note that sometimes Clp and Gurobi give slightly different results)
        begin
            # Set path to license (for those using Gurobi)
            ENV["GRB_LICENSE_FILE"] = "/Users/jimmy/gurobi.lic"
            m = Model(Gurobi.Optimizer)
        end
    end
    ######## Decision variables ########

    begin
        # first-stage decision variables
        @variable(m, PVSize >= 0); # [kW] Solar PV Power Capacity
        @variable(m, WindSize >= 0); # [kW] Wind Farm Power Capacity
        @variable(m, ESize >= 0); # [kW] Electrolyzer Power Capacity
        @variable(m, FSize >= 0); # [kW] Hydrogen Fuel Cell Capacity
        @variable(m, SSSize >= 0); # [kg] Short Term Storage Energy Capacity
        @variable(m, LSSize >= 0); # [kg] Long Term Storage Energy Capacity

    end

    begin
        # second stage decision variables
        @variable(m, PV2R[s=1:10, 1:NumTime] >= 0); # [kW] electrical power transfer from PV to residential load

        @variable(m, PV2I[s=1:10, 1:NumTime] >= 0); # [kW] electrical power transfer from PV to industrial load

        @variable(m, W2R[s=1:10, 1:NumTime] >= 0); # [kW] electrical power transfer from wind farm to residential load

        @variable(m, W2I[s=1:10, 1:NumTime] >= 0); # [kW] electrical power transfer from wind to industrial load

        @variable(m, PV2E[s=1:10, 1:NumTime] >= 0); # [kW] electrical power transfer from PV to electrolyzer

        @variable(m, W2E[s=1:10, 1:NumTime] >= 0); # [kW] electrical power transfer from wind to electrolyzer

        @variable(m, E2F[s=1:10, 1:NumTime] >= 0); # [kg] electrolyzer to fuel cell

        @variable(m, E2SS[s=1:10, 1:NumTime] >= 0); # [kg] electrolyzer to short term storage

        @variable(m, E2I[s=1:10, 1:NumTime] >= 0); # [kg] electrolyzer to industrial gas load

        @variable(m, F2R[s=1:10, 1:NumTime] >= 0); # [kW] fuel cell to residential load

        @variable(m, F2I[s=1:10, 1:NumTime] >= 0); # [kW] fuel cell to industrial load

        @variable(m, E2LS[s=1:10, 1:NumTime] >= 0); # [kg] electrolyzer to long term storage

        @variable(m, SS2F[s=1:10, 1:NumTime] >= 0); # [kg] hydrogen gas discharged from short term storage to fuel cell

        @variable(m, SS2I[s=1:10, 1:NumTime] >= 0); # [kg] hydrogen gas discharged from short term storage to industrial gas load

        @variable(m, LS2F[s=1:10, 1:NumTime] >= 0); # [kg] hydrogen liquid discharged from long term storage to fuel cell

        @variable(m, LS2I[s=1:10, 1:NumTime] >= 0); # [kg] hydrogen liquid discharged from long term storage to industrial gas load

        @variable(m, InStorageSS[s=1:10, 1:NumTime] >= 0); # [kg] Short term remaining storage

        @variable(m, InStorageLS[s=1:10, 1:NumTime] >= 0); # [kg] Long term remaining storage

        @variable(m, PV2G[s=1:10, 1:NumTime] >= 0); # [kW] electrical power transfer from PV to curtailment

        @variable(m, W2G[s=1:10, 1:NumTime] >= 0); # [kW] electrical power transfer from wind farm to curtailment
    end

    ############ Objective Functions #############
    begin
        # First-stage costs: Capital and fixed operational
        @expression(m, capital_cost, C_PV * PVSize + C_W * WindSize + C_FC * FSize + C_EL * ESize + C_C_ls * LSSize + C_C_ss * SSSize)
        @expression(m, fixed_OM_cost, C_PV_OP * PVSize + C_W_OP * WindSize)
        @expression(m, first_stage_cost, capital_cost * CRF + fixed_OM_cost)

        # Second-stage costs: Variable costs dependent on scenario-specific data
        @expression(m, second_stage_cost[s in keys(scenarios)],
            sum((InStorageSS[s, t] * C_ss + InStorageLS[s, t] * C_ls + LS2I[s, t] * C_d) for t=1:NumTime) +
            sum((E2LS[s, t] * C_g2l + (LS2F[s, t] + LS2I[s, t]) * C_l2g) for t=1:NumTime))

        # Objective function: Minimize expected total cost
        scenario_probability = 1.0 / length(scenarios)  # Adjust if different probabilities are given
        @objective(m, Min, first_stage_cost + scenario_probability * sum(second_stage_cost[s] for s in keys(scenarios)))

    end
    ############# Expressions ############

    ############# Constraints ############
    begin
        # Loop through each scenario
        for s in keys(scenarios)
            input = scenarios[s]  # Access scenario-specific input data

            # Initialization constraints
            @constraint(m, InStorageSS[s, 1] == 0.5 * SSSize);
            @constraint(m, InStorageSS[s, NumTime] >= 0.5 * SSSize);
            @constraint(m, InStorageLS[s, 1] == 0);

            # Energy balance constraints
            @constraint(m, [t=1:NumTime], input[t, 10] * PVSize * 1000 == PV2E[s, t] + PV2R[s, t] + PV2I[s, t] + PV2G[s, t]);
            @constraint(m, [t=1:NumTime], input[t, 9] * WindSize * 1000 == W2E[s, t] + W2R[s, t] + W2I[s, t] + W2G[s, t]);

            # Energy conversion and efficiency constraints
            @constraint(m, [t=1:NumTime], δt * (W2E[s, t] + PV2E[s, t]) * α_EL * k_E2H == E2F[s, t] + E2SS[s, t] + E2I[s, t] + E2LS[s, t]);
            @constraint(m, [t=1:NumTime], E2F[s, t] + SS2F[s, t] + LS2F[s, t] * β_l2g == δt * (F2R[s, t] + F2I[s, t]) / (α_FC * k_H2E));

            # Storage dynamics and leakage
            @constraint(m, [t=1:NumTime-1], InStorageSS[s, t+1] == (InStorageSS[s, t] + E2SS[s, t] - SS2I[s, t] - SS2F[s, t]) * (1-L_ss));
            @constraint(m, [t=1:NumTime-1], InStorageLS[s, t+1] == (InStorageLS[s, t] + E2LS[s, t] * β_g2l - LS2F[s, t] - LS2I[s, t]) * (1-L_ls));

            # Storage capacity limits
            @constraint(m, [t=1:NumTime], InStorageSS[s, t] <= SSSize);
            @constraint(m, [t=1:NumTime], InStorageLS[s, t] <= LSSize);

            # Demand satisfaction
            @constraint(m, [t=1:NumTime], sum(input[t, i] for i = 4:8) * 1000 == PV2R[s, t] + W2R[s, t] + F2R[s, t]);
            @constraint(m, [t=1:NumTime], sum(input[t, i] for i = 2:3) * 1000 == PV2I[s, t] + W2I[s, t] + F2I[s, t]);
            @constraint(m, [t=1:NumTime], input[t, 11] == E2I[s, t] + SS2I[s, t] + LS2I[s, t] * β_l2g * μ);

            # Capacity constraints
            @constraint(m, [t=1:NumTime], W2E[s, t] + PV2E[s, t] <= ESize);
            @constraint(m, [t=1:NumTime], (F2R[s, t] + F2I[s, t]) <= FSize);
        end
    end
    ########### Solve  ##########
    optimize!(m);
    ########### Model Results  ##########
    begin
        # Return system sizes and other scalar and time series data

        PV_Size = round(value.(PVSize), digits=2); # [kW]

        Wind_Size = round(value.(WindSize), digits=2); # [kW]

        E_Size = round(value.(ESize), digits=2); # [kW]

        F_Size = round(value.(FSize), digits=2); # [kW]

        SS_Size = round(value.(SSSize), digits=2); # [kg]

        LS_Size = round(value.(LSSize), digits=2); # [kg]

        ObjValue = objective_value(m); # [$] Levelized cost of system over lifetime

        # InStorageSS_values = [round(value(InStorageSS[t]), digits=2) for t in TIME]
        # InStorageLS_values = [round(value(InStorageLS[t]), digits=2) for t in TIME]
    end
    results_dict = Dict(
        "Costs" => ObjValue / 10^6, # Converted to millions USD for readability
        "Capacities" => Dict(
            "PV" => PV_Size,
            "Wind" => Wind_Size,
            "Electrolyzer" => E_Size,
            "Fuel Cell" => F_Size,
            "Short Term Storage" => SS_Size,
            "Long Term Storage" => LS_Size
        ),
        # "Flows" => Dict(
        #     "InStorageSS" => InStorageSS_values,
        #     "InStorageLS" => InStorageLS_values,
        #     # Include other variables similarly...
        # )
    )

    return results_dict
end

results = Optimize(15)


Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-15
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm])
Thread count: 11 physical cores, 11 logical processors, using up to 11 threads
Optimize a model with 87370 rows, 134406 columns and 331727 nonzeros
Model fingerprint: 0xa1834845
Coefficient statistics:
  Matrix range     [5e-03, 5e+03]
  Objective range  [3e-02, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 2e+05]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 12769 rows and 34685 columns
Presolve time: 0.26s
Presolved: 74601 rows, 99721 columns, 271584 nonzeros

Ordering time: 0.01s

Barrier statistics:
 Dense cols : 6
 AA' NZ     : 1.903e+05
 Factor NZ  : 1.015e+06 (roughly 80 MB of memory)
 Factor Ops : 1.492e+07 (less than 1 second per iteration)
 Threads    : 9

                  Objective                Residual
Iter       Primal          Dual 

Dict{String, Any} with 2 entries:
  "Costs"      => 8.11093
  "Capacities" => Dict("Fuel Cell"=>0.0, "Wind"=>278.01, "Electrolyzer"=>118306…

In [85]:
println(results["Capacities"])

Dict("Fuel Cell" => 0.0, "Wind" => 278.01, "Electrolyzer" => 118305.98, "Short Term Storage" => 1423.78, "PV" => 0.0, "Long Term Storage" => 0.0)


In [86]:
function optimize_and_summarize(master)
    optimize!(master)
    if termination_status(master) == MOI.OPTIMAL
        println("Optimization successful!")

        # Retrieve and print the optimal objective value
        optimal_obj_value = objective_value(master)
        println("Optimal Objective Value: ", optimal_obj_value)

        # Retrieve and print all decision variables and their optimal values
        println("Values of Decision Variables:")
        for var in all_variables(master)
            println("  ", name(var), " = ", value(var))
        end
    else
        println("Optimization failed with status: ", termination_status(master))
    end
end


optimize_and_summarize (generic function with 1 method)

In [91]:



# function benders_decomposition()
#     master = Model(Gurobi.Optimizer)
#     set_optimizer_attribute(master, "OutputFlag", 0)  # Suppress Gurobi output
#     @variable(master, PVSize >= 0)
#     @variable(master, WindSize >= 0)
#     @variable(master, ESize >= 0)
#     @variable(master, FSize >= 0)
#     @variable(master, SSSize >= 0)
#     @variable(master, LSSize >= 0)

#     # Initial placeholder for the second-stage costs (using a very loose upper bound or estimate)
#     @variable(master, θ[1:10] >= 0)

#     @expression(master, capital_cost, C_PV * PVSize + C_W * WindSize + C_FC * FSize + C_EL * ESize + C_C_ls * LSSize + C_C_ss * SSSize)
#     @expression(master, fixed_OM_cost, C_PV_OP * PVSize + C_W_OP * WindSize)
#     @objective(master, Min, capital_cost * CRF + fixed_OM_cost + sum(θ[i] for i = 1 : 10))
#     c
#     apacities = Dict()
#     results = Dict()
#     optimal_costs = []
#     max_iterations = 100

#     for iteration = 1:max_iterations
#         println("Iteration: ", iteration)
#         optimize!(master)
#         expected_cost = 0
#         cuts_added = false
        # if iteration == 1
        #     # fix first iteration initialization point
        #     # please do not change this for testing purpose
        #     PVSize, WindSize, ESize, FSize, SSSize, LSSize = [0.0, 278.01,118305.98, 0.0, 1423.78, 0.0]
        # end

#         if iteration == 2
#             break
#         end

#         for s in keys(scenarios)
#             capacities["PVSize"] = value(PVSize)
#             capacities["WindSize"] = value(WindSize)
#             capacities["ESize"] = value(ESize)
#             capacities["FSize"] = value(FSize)
#             capacities["SSSize"] = value(SSSize)
#             capacities["LSSize"] = value(LSSize)

#             sub, pv_capacity_constraints, wind_capacity_constraints, EL_energy_conversion_constraints, FC_energy_conversion_constraints, inStorageSS_dynamics_constraints, inStorageLS_dynamics_constraints,
#             inStorageSS_constraints, inStorageLS_constraints, resident_demand_constraints, industrial_demand_constraints, gas_demand_constraints, eSize_capacity_constraints, fSize_capacity_constraints = create_subproblem(s, capacities)
#             optimize!(sub)

#             if termination_status(sub) == MOI.OPTIMAL
#                 sub_cost = objective_value(sub)
#                 expected_cost += sub_cost * (1.0 / length(scenarios))  # Assuming equal probability for scenarios

#                 # Extract dual values for constraints linked to capacity variables
#                 duals = Dict()
#                 duals["PVSize"] =  [dual(con) for con in pv_capacity_constraints]
#                 duals["WindSize"] =  [dual(con) for con in wind_capacity_constraints]
#                 duals["ESize"] =  [dual(con) for con in eSize_capacity_constraints]
#                 duals["FSize"] =  [dual(con) for con in fSize_capacity_constraints]
#                 duals["SSSize"] = [dual(con) for con in inStorageSS_constraints]
#                 duals["LSSize"] =  [dual(con) for con in inStorageLS_constraints]

#                 # Create a Bender's cut
#                 cut_expr = sum(duals[key][t] * (capacities[key] - value(getfield(master, Symbol(key)))) for key in keys(duals) for t in eachindex(duals[key]))
#                 cuts_added = true
#                 println("ADDED CUT: ", cut_expr)

#             elseif termination_status(sub) == MOI.INFEASIBLE
#                 println("Subproblem $s is infeasible.")
#                 # Handling for infeasible subproblem, potentially retrieving dual rays or infeasibility certificates
#                 # Additional logic needed here 

#                 #### 1. Extract undounded dual array
#                 #### 2. Get Tx - h, formalize T and h
#             else
#                 println("Subproblem $s did not solve to optimality; status: ", termination_status(sub))
#             end
#         end

#         if cuts_added
#             push!(optimal_costs, objective_value(master))
#             if iteration > 1 && abs(optimal_costs[end] - optimal_costs[end-1]) < convergence_tolerance
#                 break
#             end
#         else
#             println("No cuts were added; might need to check subproblem formulations or solver issues.")
#         end
#     end

#     results["optimal_costs"] = optimal_costs
#     results["capacities"] = [value(PVSize), value(WindSize), value(ESize), value(FSize), value(SSSize), value(LSSize)]
#     return results
# end

# results = benders_decomposition()
# println("Optimal Costs: ", results["optimal_costs"])
# println("Optimal Capacities: ", results["capacities"])


LoadError: InterruptException:

In [145]:
using JuMP, Gurobi

# Constants for costs, capacity factors, etc.
C = Dict("PVSize" => C_PV, "WindSize" => C_W, "ESize" => C_EL, "FSize" => C_FC, "SSSize" => C_C_ss, "LSSize" => C_C_ls)
C_OP = Dict("PVSize" => C_PV_OP, "WindSize" => C_W_OP)

using JuMP, Gurobi

function create_subproblem(s, capacities)
    sub = Model(Gurobi.Optimizer)
    @variable(sub, PV2R[1:NumTime] >= 0)
    @variable(sub, PV2I[1:NumTime] >= 0)
    @variable(sub, W2R[1:NumTime] >= 0)
    @variable(sub, W2I[1:NumTime] >= 0)
    @variable(sub, PV2E[1:NumTime] >= 0)
    @variable(sub, W2E[1:NumTime] >= 0)
    @variable(sub, E2F[1:NumTime] >= 0)
    @variable(sub, E2SS[1:NumTime] >= 0)
    @variable(sub, E2I[1:NumTime] >= 0)
    @variable(sub, F2R[1:NumTime] >= 0)
    @variable(sub, F2I[1:NumTime] >= 0)
    @variable(sub, E2LS[1:NumTime] >= 0)
    @variable(sub, SS2F[1:NumTime] >= 0)
    @variable(sub, SS2I[1:NumTime] >= 0)
    @variable(sub, LS2F[1:NumTime] >= 0)
    @variable(sub, LS2I[1:NumTime] >= 0)
    @variable(sub, InStorageSS[1:NumTime] >= 0)
    @variable(sub, InStorageLS[1:NumTime] >= 0)
    @variable(sub, PV2G[1:NumTime] >= 0)
    @variable(sub, W2G[1:NumTime] >= 0)

    # Assuming scenarios is a predefined global or passed dictionary
    input = scenarios[s]  # Scenario-specific data

    # Constraints specific to scenario `s` using `capacities`
    pv_capacity_constraints = @constraint(sub, [t=1:NumTime], input[t, 10] * capacities["PVSize"] * 1000 == PV2E[t] + PV2R[t] + PV2I[t] + PV2G[t])
    wind_capacity_constraints = @constraint(sub, [t=1:NumTime], input[t, 9] * capacities["WindSize"] * 1000 == W2E[t] + W2R[t] + W2I[t] + W2G[t])

    # Energy conversion and efficiency constraints
    EL_energy_conversion_constraints = @constraint(sub, [t=1:NumTime], δt * (W2E[t] + PV2E[t]) * α_EL * k_E2H == E2F[t] + E2SS[t] + E2I[t] + E2LS[t])
    FC_energy_conversion_constraints = @constraint(sub, [t=1:NumTime], E2F[t] + SS2F[t] + LS2F[t] * β_l2g == δt * (F2R[t] + F2I[t]) / (α_FC * k_H2E))

    # Storage dynamics and leakage
    inStorageSS_dynamics_constraints = @constraint(sub, [t=1:NumTime-1], InStorageSS[t+1] == (InStorageSS[t] + E2SS[t] - SS2I[t] - SS2F[t]) * (1 - L_ss))
    inStorageLS_dynamics_constraints = @constraint(sub, [t=1:NumTime-1], InStorageLS[t+1] == (InStorageLS[t] + E2LS[t] * β_g2l - LS2F[t] - LS2I[t]) * (1 - L_ls))

    # Storage capacity limits
    inStorageSS_constraints = @constraint(sub, [t=1:NumTime], InStorageSS[t] <= capacities["SSSize"])
    inStorageLS_constraints = @constraint(sub, [t=1:NumTime], InStorageLS[t] <= capacities["LSSize"])

    # Demand satisfaction
    resident_demand_constraints = @constraint(sub, [t=1:NumTime], sum(input[t, i] for i = 4:8) * 1000 == PV2R[t] + W2R[t] + F2R[t])
    industrial_demand_constraints = @constraint(sub, [t=1:NumTime], sum(input[t, i] for i = 2:3) * 1000 == PV2I[t] + W2I[t] + F2I[t])
    gas_demand_constraints = @constraint(sub, [t=1:NumTime], input[t, 11] == E2I[t] + SS2I[t] + LS2I[t] * β_l2g * μ)

    # Capacity constraints
    eSize_capacity_constraints = @constraint(sub, [t=1:NumTime], W2E[t] + PV2E[t] <= capacities["ESize"])
    fSize_capacity_constraints = @constraint(sub, [t=1:NumTime], (F2R[t] + F2I[t]) <= capacities["FSize"])

    # Objective: Minimize operational costs for scenario `s`
    @objective(sub, Min,
        sum((InStorageSS[t] * C_ss + InStorageLS[t] * C_ls + LS2I[t] * C_d) for t=1:NumTime) +
        sum((E2LS[t] * C_g2l + (LS2F[t] + LS2I[t]) * C_l2g) for t=1:NumTime))

    return sub, Dict(
        "PVSize" => pv_capacity_constraints,
        "WindSize" => wind_capacity_constraints,
        "ESize" => eSize_capacity_constraints,
        "FSize" => fSize_capacity_constraints,
        "SSSize" => inStorageSS_constraints,
        "LSSize" => inStorageLS_constraints
    )
end

function extract_duals(constraint_refs)
    duals = Dict()
    for (key, constraints) in constraint_refs
        if !isempty(constraints) && all(isa(con, ConstraintRef) for con in constraints)
            duals[key] = [dual(con) for con in constraints]
        else
            println("Warning: Constraints for $key are not properly formatted or empty.")
        end
    end
    return duals
end
function add_optimality_cut(master, vars, θ, s, sub_cost, duals, capacities,cut_dict)
    try
        cut_expr = sum(duals[key][t] * (capacities[key] - value(vars[key])) for key in keys(duals) for t in 1:length(duals[key]))
        cut_dict[s] = sub_cost - cut_expr
        println("added cut to dict for ", s)
    catch error
        println("Failed to add optimality cut for scenario $s: $error")
    end
end



add_optimality_cut (generic function with 3 methods)

In [147]:
using JuMP, Gurobi

function benders_decomposition()
    master = Model(Gurobi.Optimizer)
    set_optimizer_attribute(master, "OutputFlag", 0)  # Suppress Gurobi output
    vars = Dict(
        "PVSize" => @variable(master, PVSize >= 0),
        "WindSize" => @variable(master, WindSize >= 0),
        "ESize" => @variable(master, ESize >= 0),
        "FSize" => @variable(master, FSize >= 0),
        "SSSize" => @variable(master, SSSize >= 0),
        "LSSize" => @variable(master, LSSize >= 0)
    )
    
    @variable(master, θ[1:10] >= 0)
    @expression(master, capital_cost, sum(vars[key] * C[key] for key in keys(vars)))
    @expression(master, fixed_OM_cost, sum(vars[key] * C_OP[key] for key in keys(vars) if haskey(C_OP, key)))
    @objective(master, Min, capital_cost * CRF + fixed_OM_cost + sum(θ))

    max_iterations = 100
    optimal_costs = []

    # Manually set initial capacities directly, and use them in the first subproblem iteration
    capacities = Dict(
        "PVSize" => 0.0,
        "WindSize" => 278.01,
        "ESize" => 118305.98,
        "FSize" => 0.0,
        "SSSize" => 1423.78,
        "LSSize" => 0.0
    )

    for iteration in 1:max_iterations
        println("------ Iteration: ", iteration)
        optimize!(master)
        println(capacities)
        
        if termination_status(master) != MOI.OPTIMAL
            println("Master problem not solved to optimality.")
            break
        end
        

        if iteration > 1
            capacities = Dict(key => value(vars[key]) for key in keys(vars))  # Update capacities based on optimization results
            print("Capacities for iteration ", iteration)
            println(capacities)
        end

        if iteration == 10
            break
        end

        expected_cost = 0
        cuts_added = false

        cut_dict = Dict()

        for s in keys(scenarios)
            
            sub, constraints = create_subproblem(s, capacities)
            optimize!(sub)
            println("--- status for ", s)
            println(termination_status(sub))
            
            
            if termination_status(sub) == MOI.OPTIMAL
                
                sub_cost = objective_value(sub)
                expected_cost += sub_cost * (1.0 / length(scenarios))
                
                duals = extract_duals(constraints)
                add_optimality_cut(master, vars, θ, s, sub_cost, duals, capacities,cut_dict)
                print(cut_dict)
                cuts_added = true
            else
                # need to implement feasibility cut
                println("Subproblem $s did not solve to optimality; status: ", termination_status(sub))
            end
            

        end

        # this loop is only for optimality cuts for now.
        println("--------LOOP here!")
        for s in keys(scenarios)
            try
                @constraint(master, θ[s] >= cut_dict[s])
            catch error
                print("Passed scenario ", s, "because of no feasibility cut.")
            end
        end
        println("--------LOOP DONE")

        if iteration > 1
            push!(optimal_costs, objective_value(master))
            if cuts_added && abs(optimal_costs[end] - optimal_costs[end-1]) < 1e-4  # Convergence check
                println("Convergence achieved.")
                break
            end
        end
    end

    return optimal_costs, Dict(key => value(vars[key]) for key in keys(vars))
end

# Example usage
results, capacities = benders_decomposition()
println("Optimal Costs: ", results)
println("Optimal Capacities: ", capacities)


Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-15
------ Iteration: 1
Dict("LSSize" => 0.0, "PVSize" => 0.0, "FSize" => 0.0, "WindSize" => 278.01, "SSSize" => 1423.78, "ESize" => 118305.98)
Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-15
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm])
Thread count: 11 physical cores, 11 logical processors, using up to 11 threads
Optimize a model with 8734 rows, 13440 columns and 29558 nonzeros
Model fingerprint: 0xc194d528
Coefficient statistics:
  Matrix range     [5e-03, 1e+00]
  Objective range  [3e-01, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 1e+06]
Presolve removed 6722 rows and 10086 columns
Presolve time: 0.01s
Presolved: 2012 rows, 3354 columns, 6706 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.5222231e+01   4.055905e+03   0.000000e+00      0s
    1010    6.0011446e+04   0.000000e+

In [113]:
capacities

Dict{String, Float64} with 6 entries:
  "LSSize"   => 0.0
  "PVSize"   => 0.0
  "FSize"    => 0.0
  "WindSize" => 0.0
  "SSSize"   => 0.0
  "ESize"    => 0.0